# [DCC 030] Aprendizado Profundo para Processamento de Linguagem Natural: Trabalho Prático 2
__Aluno:__
- Eduardo Villani de Carvalho Filho - 2015104008
---
# [Parte 1] O trabalho

## Bibliotecas que serão utilizadas

Aqui é definido todas as bibliotecas que serão utilizadas ao longo deste trabalho.

In [1]:
import multiprocessing
import pathlib
import nltk
import ssl
import numpy as np
import joblib

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import Embedding, LSTM, TimeDistributed, Dense
from tensorflow.python.keras.models import Sequential
from datetime import datetime
from timeit import default_timer as timer
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from typing import Union
from tqdm.notebook import tqdm

## Eventual problema de SSL

Há um pequeno problema no sistema de download da biblioteca utilizada, por isso devemos rodar a
linha abaixo para configurar o certificado ssl do computador.

In [2]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

## Introdução

Este trabalho tem como objetivo pôr em prática o que foi visto em sala de aula do conceito de
POS Tagging. O POS Tagging tem por objetivo classificar as palavras em classes gramaticais, isto
é, pegando uma frase genérica como: "Eu lavei meu carro ontem", o POS Tagging teria por objetivo
saber classificar "Eu" como um Pronome Pessoa, "lavei" como um verbo, "meu" como um pronome
possessivo, "carro" como um substantivo e "ontem" como um advérbio de tempo, além de outras
classificações possíveis, como "meu carro ontem" como um objeto direto.

Para a elaboração deste trabalhos, iremos utilizar o corpus Mac Morpho (colocar link aqui),
o qual contém classificação de sentenças em português brasileiro feita pela USP. Pegando um exemplo prático
utilizando a biblioteca do Python NLTK:

In [3]:
nltk.download('mac_morpho')
mac_morpho = nltk.corpus.mac_morpho
for word_tagged in mac_morpho.tagged_sents()[0]:
    print(word_tagged)

del word_tagged

('Jersei', 'N')
('atinge', 'V')
('média', 'N')
('de', 'PREP')
('Cr$', 'CUR')
('1,4', 'NUM')
('milhão', 'N')
('em', 'PREP|+')
('a', 'ART')
('venda', 'N')
('de', 'PREP|+')
('a', 'ART')
('Pinhal', 'NPROP')
('em', 'PREP')
('São', 'NPROP')
('Paulo', 'NPROP')


[nltk_data] Downloading package mac_morpho to
[nltk_data]     /Users/eduardovillani/nltk_data...
[nltk_data]   Package mac_morpho is already up-to-date!


Como podemos notar, o nltk apresenta a classificação de cada palavra e ainda a junção de dois tipos
diferentes para formar um só, como de + a formando da, como uma preposição e um artigo.

Ao longo deste trabalho, iremos ver o funcionamento do NLTK (para se familiarizar com a biblioteca e o POS Tagging),
passando por uma avaliação dos classificadores do NLTK e finalizando momontando um modelo usando XYZ
para comparação com estes classificadores.

# [Parte 2] Conhecendo o nltk e o MacMorphos

Aqui iremos conhecer um pouco da bilioteca nltk e do MacMorphos, para então validar
um modelo de classificação.

In [4]:
def most_commom_tags(corpus, n):
    def simplify_tag(t):
        """Retirado de: http://www.nltk.org/howto/portuguese_en.html"""
        if "+" in t:
            return t[t.index("+")+1:]
        else:
            return t

    tags = [simplify_tag(tag) for (word,tag) in corpus.tagged_words()]
    fd_tags =  nltk.FreqDist(tags)
    for tag in list(fd_tags.keys())[:n]:
        print(tag)
    del fd_tags
    del tags

In [5]:
n = 10
print(f"As {n} tags mais comuns em pt-BR:")
most_commom_tags(mac_morpho, n)


As 10 tags mais comuns em pt-BR:
N
V
PREP
CUR
NUM

ART
NPROP
PROADJ
,


In [7]:
fd = nltk.FreqDist(mac_morpho.tagged_words())

In [8]:
n = 10
print(f"As {n} palavras mais comuns em pt-BR:\n{fd.most_common(n)}")
del fd

As 10 palavras mais comuns em pt-BR:
[((',', ','), 68491), (('o', 'ART'), 51854), (('a', 'ART'), 46588), (('de', 'PREP'), 43093), (('de', 'PREP|+'), 39427), (('e', 'KC'), 21267), (('"', '"'), 21069), (('em', 'PREP|+'), 18586), (('os', 'ART'), 14541), (('em', 'PREP'), 11826)]


# [PARTE 3] Testando modelos de determinação de Tags com o nltk

Iremos avaliar alguns classificadores do nltk com algumas tags, para então
usar eles como base para validar o modelo aqui desenvolvido.

## Dados de Treino e Dados de Teste

Vamos usar 80% dos dados para testes e 20% para treino.

In [9]:
k = 0.8
data = mac_morpho.tagged_sents()
tot = len(data)
tot_train_samples = int(np.ceil(tot*k))

train_data = data[:tot_train_samples]
test_data = data[tot_train_samples:]

del data, k, tot, tot_train_samples

## Fábricas de Modelos

Para facilitar o desenvolvimento do trabalho, será feito uma fábrica de modelos para criação e gerenciamento dos mesmos.
Abaixo explicamos a função de cada classe criada.

### Tipos de Modelos

Aqui definimos os tipos de modelos que serão usados para avaliar a acurácia.

In [10]:
class ModelType:
    TAGGER = 'TAGGER'
    AFFIX2 = 'AFFIX2'
    AFFIX3= 'AFFIX3'
    AFFIX4 = 'AFFIX4'
    AFFIX5 = 'AFFIX5'
    AFFIX6 = 'AFFIX6'
    UNIGRAM = 'UNIGRAM'
    BIGRAM = 'BIGRAM'
    TRIGRAM = 'TRIGRAM'
    BRILL_TAGGER = 'BRILL_TAGGER'
    NAIVES_BAYES = 'NAIVES_BAYES'

### Tagger de Modelos

Uma classe básica que contém informações dos modelos (valor padrão no texto, modelos, classificador usado,
acurácia e se a acurácia está em porcentual ou em valores unitário). Foi feito uma subclasse para
controle dos modelos.

In [11]:
class Tagger:
    class ModelDict:
        def __init__(self, name: str, classifier):
            self._data = {
                'name': name,
                'accuracy': None,
                'accuracy_type': "unit",
                'model': None,
                'classifier': classifier
            }

        @property
        def name(self) -> str:
            return self['name']

        @property
        def accuracy(self) -> Union[float, None]:
            return self['accuracy']

        @property
        def model(self):
            return self['model']

        @property
        def classifier(self):
            return self['classifier']

        def __str__(self):
            return str(self._data)

        def __getitem__(self, item: str):
            return self._data[item]

        def __setitem__(self, item: str, value):
            self._data[item] = value

        __repr__ = __str__

    def __init__(self, value: str):
        self._data = {
            "value": value,
            'models': {
                ModelType.TAGGER: Tagger.ModelDict(ModelType.TAGGER, nltk.DefaultTagger),
                ModelType.AFFIX2: Tagger.ModelDict(ModelType.AFFIX2, nltk.AffixTagger),
                ModelType.AFFIX3: Tagger.ModelDict(ModelType.AFFIX3, nltk.AffixTagger),
                ModelType.AFFIX4: Tagger.ModelDict(ModelType.AFFIX4, nltk.AffixTagger),
                ModelType.AFFIX5: Tagger.ModelDict(ModelType.AFFIX5, nltk.AffixTagger),
                ModelType.AFFIX6: Tagger.ModelDict(ModelType.AFFIX6, nltk.AffixTagger),
                ModelType.UNIGRAM: Tagger.ModelDict(ModelType.UNIGRAM, nltk.UnigramTagger),
                ModelType.BIGRAM: Tagger.ModelDict(ModelType.BIGRAM, nltk.BigramTagger),
                ModelType.TRIGRAM: Tagger.ModelDict(ModelType.TRIGRAM, nltk.TrigramTagger),
                ModelType.NAIVES_BAYES: Tagger.ModelDict(ModelType.NAIVES_BAYES, nltk.ClassifierBasedPOSTagger)
    }
        }

    @property
    def value(self) -> str:
        return self['value']

    @property
    def models(self)->dict:
        return self['models']


    def load_accuracies(self, name):
        import json
        with open('accuracy.json', 'r') as fp:
            saved_accuracies = json.load(fp)
        fp.close()
        with tqdm(self.models, position=1, desc=f'Loading accuracy models for {name}') as inner_for:
            for m in inner_for:
                self['models'][m]['accuracy'] = saved_accuracies[name][m]

    def evaluate_models(self, test_data, name):
        with tqdm(self.models, position=1, desc=f'Evaluating models for {name}') as inner_for:
            for m in inner_for:
                model = self.models[m]
                accuracy_model = model.accuracy
                if accuracy_model is not None and isinstance(accuracy_model, float):
                    pass
                m_model = model.model.evaluate(test_data)
                self['models'][m]['accuracy'] = m_model


    def generate_models(self, train_data, name):
        with tqdm(self.models, position=1, desc=f'Generating models for {name}') as inner_for:
            for m in inner_for:
                current_model = self['models'][m]['model']
                pathlib.Path("models").mkdir(parents=True, exist_ok=True)
                file_name = f"POS_tagger_{self['value']}_{self['models'][m]['name']}"
                file_path = f"models/{file_name}.pkl"
                tagger = None
                if current_model is None:
                    try:
                        self['models'][m]['model'] = joblib.load(file_path)
                    except FileNotFoundError:
                        if m == ModelType.TAGGER:
                            tagger = self['models'][m]['classifier'](self['value'])
                        elif m == ModelType.NAIVES_BAYES:
                            tagger = self['models'][m]['classifier'](train=train_data)
                        elif m == ModelType.BRILL_TAGGER:
                            backoff = self['models'][ModelType.TRIGRAM]['model']
                            tagger = self['models'][m]['classifier'](backoff, nltk.brill.fntbl37(), trace=True)
                            tagger = tagger.train(train_data)
                        elif m == ModelType.AFFIX2:
                            backoff = self['models'][ModelType.TAGGER]['model']
                            tagger = self['models'][m]['classifier'](train_data,affix_length=-2, backoff=backoff)
                        elif m == ModelType.AFFIX3:
                            backoff = self['models'][ModelType.AFFIX2]['model']
                            tagger = self['models'][m]['classifier'](train_data,affix_length=-3, backoff=backoff)
                        elif m == ModelType.AFFIX4:
                            backoff = self['models'][ModelType.AFFIX3]['model']
                            tagger = self['models'][m]['classifier'](train_data,affix_length=-4, backoff=backoff)
                        elif m == ModelType.AFFIX5:
                            backoff = self['models'][ModelType.AFFIX4]['model']
                            tagger = self['models'][m]['classifier'](train_data,affix_length=-5, backoff=backoff)
                        elif m == ModelType.AFFIX6:
                            backoff = self['models'][ModelType.AFFIX5]['model']
                            tagger = self['models'][m]['classifier'](train_data,affix_length=-6, backoff=backoff)
                        elif m == ModelType.UNIGRAM:
                            backoff = self['models'][ModelType.AFFIX6]['model']
                            tagger = self['models'][m]['classifier'](train_data, backoff=backoff)
                        elif m == ModelType.BIGRAM:
                            backoff = self['models'][ModelType.UNIGRAM]['model']
                            tagger = self['models'][m]['classifier'](train_data, backoff=backoff)
                        elif m == ModelType.TRIGRAM:
                            backoff = self['models'][ModelType.BIGRAM]['model']
                            tagger = self['models'][m]['classifier'](train_data, backoff=backoff)
                        if tagger is not None:
                            self['models'][m]['model'] = tagger
                            joblib.dump(tagger, file_path)

    def __getitem__(self, item):
        return self._data[item]

    def __setitem__(self, item, value):
        self._data[item] = value

    def __str__(self):
        return str(self._data)

    __repr__ = __str__

### Dícionarios de Tags

Classe que contém todas as tags utilizadas e com possibilidade de gerar todas (somente algumas), além de avaliá-lo.

In [17]:
class TagsDict:
    def __init__(self, train_data, test_data):
        self._train_data = train_data
        self._test_data = test_data
        self._data = {
            "ADJETIVO": Tagger("ADJ"),
            "ADVÉRBIO": Tagger("ADV"),
            "ADVÉRBIO CONECTIVO SUBORDINATIVO": Tagger("ADV-KS"),
            "ADVÉRBIO RELATIVO SUBORDINATIVO": Tagger("ADV-KS-REL"),
            "ARTIGO": Tagger("ART"),
            "CONJUNÇÃO COORDENATIVA": Tagger("KC"),
            "CONJUNÇÃO SUBORDINATIVA": Tagger("KS"),
            "INTERJEIÇÃO": Tagger("IN"),
            "NOME": Tagger("N"),
            "NOME PRÓPRIO": Tagger("NPROP"),
            "NUMERAL": Tagger("NUM"),
            "PARTICÍPIO": Tagger("PCP"),
            "PALAVRA DENOTATIVA": Tagger("PDEN"),
            "PREPOSIÇÃO": Tagger("PREP"),
            "PRONOME ADJETIVO": Tagger("PROADJ"),
            "PRONOME CONECTIVO SUBORDINATIVO": Tagger("PRO-KS"),
            "PRONOME PESSOAL": Tagger("PROPESS"),
            "PRONOME RELATIVO CONECTIVO SUBORDINATIVO": Tagger("PRO-KS-REL"),
            "PRONOME SUBSTANTIVO": Tagger("PROSUB"),
            "VERBO": Tagger("V"),
            "VERBO AUXILIAR": Tagger("VAUX"),
            "SÍMBOLO DE MOEDA CORRENTE": Tagger("CUR")
        }

    def train_all_models(self):
        with tqdm(self._data, position=0, desc="Model Generating...") as main_for:
            for t in main_for:
                self._data[t].generate_models(self._train_data, t)

    def evaluate_all_models(self, force_evaluate=False):
        import json
        if force_evaluate:
            with tqdm(self._data, position=0, desc="Model Evaluating...") as main_for:
                for t in main_for:
                    self._data[t].evaluate_models(self._test_data, t)
            k = {}
            for t in self._data:
                tag =self._data[t]
                if t not in k:
                    k[t] = {}
                for m in tag.models:
                    model = tag.models[m]
                    if m not in k[t]:
                        k[t][m] = model.accuracy

            with open('accuracy.json', 'w') as fp:
                json.dump(k, fp)
        else:
            try:
                self.load_accuracies_for_all_models()
            except Exception:
                self.train_all_models(True)

    def load_accuracies_for_all_models(self):
        with tqdm(self._data, position=0, desc="Model Evaluating...") as main_for:
            for t in main_for:
                self._data[t].load_accuracies(t)

    def print_accuracies(self):
        for t in self._data:
            tag =self._data[t]
            for m in tag.models:
                model = tag.models[m]
                value = model.accuracy * 100
                print("Acurácia para {} no modelo {}: {:.2f}%".format(t, m, value))
            print("\n")

    def __str__(self):
        return str(self._data)
    __repr__ = __str__

## Conhecendo os classificadores do NLTK

Aqui faremos uma breve explicação de como cada classificador do NLTK funciona para melhor entendimento.

### Default Tagger

Primeiramente iremos utilizar o Default Tagger do nltk para classificar o texto aleatoriamente em alguma
classe gramatical. Ela servirá como um valor base para demais métodos que serão usados. Como padrão, iremos
usar a tag N, de nome. A escolha é puramente por ser a tag mais comum, mas qualquer outra tag poderia
ser usada.


### Affix

O AffixTagger se baseia em sufixos e prefixos. A língua portuguesa tem uma forma relação
entre os sufixos das palavras e sua classe. À exemplo: palavras terminar em -er, -ir ou -ar são em
sua maioria verbos. Palavras terminadas em -mente são advérbios de modo. O limite colocado foi 6,
justamente pois -mente é o maior sufixo existente em portugues, portando de 5 para 6 não haverá grandes
diferenças. Os prefixos foram ignorados por não apresentarem muita relação.

### Unigram
O UnigramTagger considera cada palavra de uma vez e determina o contexto da palavra.

### Bigram

O BigramTagger funciona como o Unigram, mas considera o conjunto de duas palavras: "Eu amo batata", os bigramas
são "eu amo" e "amo batata"

### Trigram
O TrigramTagger segue a mesma lógica do Unigram e do Trigram.

### Naives-Bayes

É um classificador baseado na teoria d Naives-Bayes. Basicamente:

\begin{equation}
P(label|features) = \frac{P(label) * P(features|label)}{|P(features)}
\end{equation}


## Gerando os modelos

Aqui iremos gerar todos os modelos para futura avaliação.

In [18]:
tags_dict = TagsDict(test_data=test_data, train_data=train_data)
tags_dict.train_all_models()

Model Generating...:   0%|          | 0/22 [00:00<?, ?it/s]

Generating models for ADJETIVO:   0%|          | 0/10 [00:00<?, ?it/s]

Generating models for ADVÉRBIO:   0%|          | 0/10 [00:00<?, ?it/s]

Generating models for ADVÉRBIO CONECTIVO SUBORDINATIVO:   0%|          | 0/10 [00:00<?, ?it/s]

Generating models for ADVÉRBIO RELATIVO SUBORDINATIVO:   0%|          | 0/10 [00:00<?, ?it/s]

...

Generating models for PRONOME RELATIVO CONECTIVO SUBORDINATIVO:   0%|          | 0/10 [00:00<?, ?it/s]

Generating models for PRONOME SUBSTANTIVO:   0%|          | 0/10 [00:00<?, ?it/s]

Generating models for VERBO:   0%|          | 0/10 [00:00<?, ?it/s]

Generating models for VERBO AUXILIAR:   0%|          | 0/10 [00:00<?, ?it/s]

Generating models for SÍMBOLO DE MOEDA CORRENTE:   0%|          | 0/10 [00:00<?, ?it/s]

## Acurácias

Aqui iremos avaliar a acurácias de todos os modelos por meio do métodos .evalute() de todos os classificadores
do nltk. Em seguida, iremos compará-los.


In [19]:
tags_dict.evaluate_all_models()

Model Evaluating...:   0%|          | 0/22 [00:00<?, ?it/s]

Loading accuracy models for ADJETIVO:   0%|          | 0/10 [00:00<?, ?it/s]

Loading accuracy models for ADVÉRBIO:   0%|          | 0/10 [00:00<?, ?it/s]

Loading accuracy models for ADVÉRBIO CONECTIVO SUBORDINATIVO:   0%|          | 0/10 [00:00<?, ?it/s]

Loading accuracy models for ADVÉRBIO RELATIVO SUBORDINATIVO:   0%|          | 0/10 [00:00<?, ?it/s]

...

Loading accuracy models for PRONOME RELATIVO CONECTIVO SUBORDINATIVO:   0%|          | 0/10 [00:00<?, ?it/s]

Loading accuracy models for PRONOME SUBSTANTIVO:   0%|          | 0/10 [00:00<?, ?it/s]

Loading accuracy models for VERBO:   0%|          | 0/10 [00:00<?, ?it/s]

Loading accuracy models for VERBO AUXILIAR:   0%|          | 0/10 [00:00<?, ?it/s]

Loading accuracy models for SÍMBOLO DE MOEDA CORRENTE:   0%|          | 0/10 [00:00<?, ?it/s]

In [20]:
tags_dict.load_accuracies_for_all_models()

Model Evaluating...:   0%|          | 0/22 [00:00<?, ?it/s]

Loading accuracy models for ADJETIVO:   0%|          | 0/10 [00:00<?, ?it/s]

Loading accuracy models for ADVÉRBIO:   0%|          | 0/10 [00:00<?, ?it/s]

Loading accuracy models for ADVÉRBIO CONECTIVO SUBORDINATIVO:   0%|          | 0/10 [00:00<?, ?it/s]

Loading accuracy models for ADVÉRBIO RELATIVO SUBORDINATIVO:   0%|          | 0/10 [00:00<?, ?it/s]

...

Loading accuracy models for PRONOME RELATIVO CONECTIVO SUBORDINATIVO:   0%|          | 0/10 [00:00<?, ?it/s]

Loading accuracy models for PRONOME SUBSTANTIVO:   0%|          | 0/10 [00:00<?, ?it/s]

Loading accuracy models for VERBO:   0%|          | 0/10 [00:00<?, ?it/s]

Loading accuracy models for VERBO AUXILIAR:   0%|          | 0/10 [00:00<?, ?it/s]

Loading accuracy models for SÍMBOLO DE MOEDA CORRENTE:   0%|          | 0/10 [00:00<?, ?it/s]

In [21]:
tags_dict.print_accuracies()

Acurácia para ADJETIVO no modelo TAGGER: 5.33%
Acurácia para ADJETIVO no modelo AFFIX2: 26.50%
Acurácia para ADJETIVO no modelo AFFIX3: 31.45%
Acurácia para ADJETIVO no modelo AFFIX4: 33.88%
Acurácia para ADJETIVO no modelo AFFIX5: 35.46%
Acurácia para ADJETIVO no modelo AFFIX6: 35.93%
Acurácia para ADJETIVO no modelo UNIGRAM: 83.72%
Acurácia para ADJETIVO no modelo BIGRAM: 85.20%
Acurácia para ADJETIVO no modelo TRIGRAM: 85.22%
Acurácia para ADJETIVO no modelo NAIVES_BAYES: 83.97%


Acurácia para ADVÉRBIO no modelo TAGGER: 3.11%
Acurácia para ADVÉRBIO no modelo AFFIX2: 27.64%
Acurácia para ADVÉRBIO no modelo AFFIX3: 32.58%
Acurácia para ADVÉRBIO no modelo AFFIX4: 35.01%
Acurácia para ADVÉRBIO no modelo AFFIX5: 36.59%
Acurácia para ADVÉRBIO no modelo AFFIX6: 37.06%
Acurácia para ADVÉRBIO no modelo UNIGRAM: 83.72%
Acurácia para ADVÉRBIO no modelo BIGRAM: 85.20%
Acurácia para ADVÉRBIO no modelo TRIGRAM: 85.22%
Acurácia para ADVÉRBIO no modelo NAIVES_BAYES: 83.97%


Acurácia para ADVÉRBIO

# [Parte 4] Montando um modelo

Aqui será desenvolvido um modelo de ML usando uma LSTM junto a embbedings.
A ideia é compará-la em percentual de acerto com os classificadores padrão do
nltk.

## Pré-processando os dados

O primeiro passo a se fazer é processar os dados. Por isso, iremos iterar em todas as
frases presentes no Mac Morpho e, dentro delas, iremos separar cada item em
palavra e em classificação. Com isso, teremos 24 tags ao final (25 na realidade, mas
será explicado logo adiante porque).

Algumas tags tiveram que ser arrumadas. Tínhamos pontuações jogadas com classificação
própria, o que aumentava o número de tags para mais de 80. Outro fator foi separar
algumas preposições (como da em de + a). Além disso, havia uma tag genérica $ que
foi ignorada, dado que ela não parecia ter significado nenhum.

Feito essa extracção de entradas e saídas (palavras são entradas, tags são saídas),
foi usado um tokenizador para dar valores números para facilitar na classificação.

Passamos por um paddings para padronizar as entradas. Para isso, considerou-se
o maior valor de frase como sendo 248, o que é menor do que 99% das frases do
Mac Morphos. Somente 14 frases tem mais de 248 palavras.

Finalmente, terminamos fazendo um One-hot encode com a saída, pois é assim que
a LSTM precisa trabalhar em cima dos dados de treino e classificação.


### Clase de Pré-processamento

Para facilitar todo esse trabalho e acesso aos dados, foi criado uma classe para guardar
e gerar os dados pré-processados.

In [42]:
class DataExtractor:
    def __init__(self, data, max_length=None, verbose=0):
        start = timer()
        if verbose:
            print(f'{datetime.now()} - Starting Data Extraction')
        self._data = data
        self._X, self._Y, self._word2index, self._tag2index, self._max_length = self._separate_mac_morpho_in_x_and_y()
        # self._Y, self._X, self._max_length = MacMorphoReader()
        if verbose == 2:
            print("{} - Total number of tagged sentences: {}".format(datetime.now(), len(self._X)))
            # print("{} - Vocabulary size: {}".format(datetime.now(), len(self._word2index)))
            # print("{} - Total number of tags: {}".format(datetime.now(), len(self._tag2index)))
            print("{} - Biggest Sentence: {}".format(datetime.now(), self._max_length))

        self._X_encoded, self._X_tokenizer = self._tokeninze_data(self._X)
        self._Y_encoded, self._Y_tokenizer = self._tokeninze_data(self._Y)

        self._word2index = self._X_tokenizer.index_word
        self._tag2index = self._Y_tokenizer.index_word

        different_length = [1 if len(input) != len(output) else 0 for input, output in
                            zip(self._X_encoded, self._Y_encoded)]
        if verbose == 2:
            print("{} - Vocabulary size: {}".format(datetime.now(), len(self._word2index)))
            print("{} - Total number of tags: {}".format(datetime.now(), len(self._tag2index)))
            print(
                "{} - {} sentences have disparate input-output lengths.".format(datetime.now(), sum(different_length)))
        if max_length is not None:
            self._max_length = max_length
        self._X_padded, self._Y_padded = self._padding_data(self._X_encoded, max_length), self._padding_data(
            self._Y_encoded, max_length)

        self._tag2index.update({0: 'pad'})
        self._word2index.update({0: 'pad'})

        self._X_tokenizer.word_index.update({'pad': 0})
        self._Y_tokenizer.word_index.update({'pad': 0})

        self._Y_one_hot = to_categorical(self._Y_padded)
        end = timer()
        if verbose:
            print(f'{datetime.now()} - Finishing Data Extraction. Took {round(end - start, 2)}s')

    @property
    def word2index(self):
        return self._word2index

    @property
    def tag2index(self):
        return self._tag2index

    @property
    def X(self):
        return self._X

    @property
    def word_tokenizer(self):
        return self._X_tokenizer

    @property
    def tag_tokenizer(self):
        return self._Y_tokenizer

    @property
    def Y_encoded(self):
        return self._Y_encoded

    @property
    def Y_one_hot(self):
        return self._Y_one_hot

    @property
    def X_encoded(self):
        return self._X_encoded

    @property
    def Y_padded(self):
        return self._Y_padded

    @property
    def X_padded(self):
        return self._X_padded

    @property
    def Y(self):
        return self._Y

    @staticmethod
    def _padding_data(data, maxlen):
        return pad_sequences(data, maxlen=maxlen, padding="pre", truncating="post")

    @staticmethod
    def _tokeninze_data(input):
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(input)
        return tokenizer.texts_to_sequences(input), tokenizer

    def _separate_mac_morpho_in_x_and_y(self):
        X, Y = [], []
        words, tags = set([]), set([])
        MAX_SEQ_LENGTH = 0
        for sentence in self._data:
            X_sentence = []
            Y_sentence = []
            count = 0
            for entity in sentence:
                word = entity[0]
                tag: str
                tag = entity[1]
                if tag.find("|") != -1:
                    tag = tag[:tag.find('|')]
                if tag in ['NPRO', 'PROP']:
                    tag = 'NPROP'
                if tag in ',().!?"-:;[]\'/...=(())`':
                    tag = "PU"
                if tag in '$':
                    continue

                words.add(word.lower())
                tags.add(tag)
                X_sentence.append(word)
                Y_sentence.append(tag)
                count = count + 1

            if count > MAX_SEQ_LENGTH:
                MAX_SEQ_LENGTH = len(sentence)

            if X_sentence or Y_sentence:
                X.append(X_sentence)
                Y.append(Y_sentence)

        word2index = {w: i + 1 for i, w in enumerate(list(words))}
        word2index['PAD'] = 0

        tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
        tag2index['PAD'] = 0
        return X, Y, word2index, tag2index, MAX_SEQ_LENGTH




### Pré-processando os dados

In [43]:
max_length = 248
data_extractor = DataExtractor(mac_morpho.tagged_sents(), max_length=max_length, verbose=2)


2021-03-17 12:21:18.572988 - Starting Data Extraction
2021-03-17 12:21:37.454606 - Total number of tagged sentences: 51397
2021-03-17 12:21:37.454767 - Biggest Sentence: 617
2021-03-17 12:22:09.399244 - Vocabulary size: 59910
2021-03-17 12:22:09.399570 - Total number of tags: 23
2021-03-17 12:22:09.399588 - 0 sentences have disparate input-output lengths.
2021-03-17 12:22:12.109124 - Finishing Data Extraction. Took 53.54s


## Gerando os Embedding

Poderíamos jogar os dados números crus em cima de algum modelo, mas para tentar verificar uma
maior acurácia na tentativa de levar-se em conta o contexto de cada palavras (pois tem
muitas em que o contexto varia sua classificação), foi gerado um embeddings usando a
biblioteca gensim e o Mac Morphos.

## Classe de geração do modelo

In [44]:
class Word2VecModel:
    def __new__(cls, text, embedding_size, file_path):
        start = timer()
        print(f'{datetime.now()} - Generating Embeddings')
        try:
            model = Word2Vec.load(file_path)
        except Exception:
            cores = multiprocessing.cpu_count()
            hyperparameters = {
                "min_count": 5,
                "window": 2,
                "size": embedding_size,
                "sample": 6e-5,
                "alpha": 0.03,
                "min_alpha": 0.0007,
                "negative": 20,
                "workers": cores - 1,
                "iter": 20
            }

            model = Word2Vec(
                text,
                sg=2,
                **hyperparameters
            )
            model.save(file_path)
        end = timer()
        print(f'{datetime.now()} - Finish Generating Embeddings. Took {round(end - start, 2)}s')
        return model

In [45]:
embedding_size = 128
vocabulary_size = len(data_extractor.word2index)
word2vec = Word2VecModel(
    data_extractor.X,
    embedding_size,
    'embedding/word2vec_mac_morphos_model_3'
)

2021-03-17 12:22:40.815055 - Generating Embeddings
2021-03-17 12:22:41.152382 - Finish Generating Embeddings. Took 0.34s


## Dados de teste, treino e validação


Outra classe construida como apoio é a de gerar os dados de teste, validação e de treino.

In [46]:
class TrainTestAndValidation:
    def __init__(self, X, Y, test_size, valid_size):
        TEST_SIZE = test_size
        X_train, X_test, Y_train, Y_test = train_test_split(
            X,
            Y,
            test_size=TEST_SIZE,
            random_state=4
        )
        VALID_SIZE = valid_size
        X_train, X_validation, Y_train, Y_validation = train_test_split(
            X_train,
            Y_train,
            test_size=VALID_SIZE,
            random_state=4
        )
        print(f'{datetime.now()} - TRAINING DATA')
        print('Shape of input sequences: {}'.format(X_train.shape))
        print('Shape of output sequences: {}'.format(Y_train.shape))
        print("-" * 50)
        print(f'{datetime.now()} - VALIDATION DATA')
        print('Shape of input sequences: {}'.format(X_validation.shape))
        print('Shape of output sequences: {}'.format(Y_validation.shape))
        print("-" * 50)
        print(f'{datetime.now()} - TESTING DATA')
        print('Shape of input sequences: {}'.format(X_test.shape))
        print('Shape of output sequences: {}'.format(Y_test.shape))

        self._X_train, self._X_test, self._Y_train, self._Y_test = X_train, X_test, Y_train, Y_test
        self._X_validation, self._Y_validation = X_validation, Y_validation

    @property
    def X_train(self):
        return self._X_train

    @property
    def X_test(self):
        return self._X_test

    @property
    def X_validation(self):
        return self._X_validation

    @property
    def Y_train(self):
        return self._Y_train

    @property
    def Y_test(self):
        return self._Y_test

    @property
    def Y_validation(self):
        return self._Y_validation


## Separandos os dados

In [47]:
train_test_and_validation = TrainTestAndValidation(data_extractor.X_padded, data_extractor.Y_one_hot, 0.2, 0.2)

2021-03-17 12:23:02.276875 - TRAINING DATA
Shape of input sequences: (32893, 248)
Shape of output sequences: (32893, 248, 24)
--------------------------------------------------
2021-03-17 12:23:02.278139 - VALIDATION DATA
Shape of input sequences: (8224, 248)
Shape of output sequences: (8224, 248, 24)
--------------------------------------------------
2021-03-17 12:23:02.278269 - TESTING DATA
Shape of input sequences: (10280, 248)
Shape of output sequences: (10280, 248, 24)


## Modelo

O Modelo escolhido foi uma LSTM com camada softmax e os embbedings.

In [48]:
class LSTMModel:
    def __new__(cls, vocabulary_size, embedding_size, max_seq_length, embedding_weights, num_classes):
        VOCABULARY_SIZE = vocabulary_size
        EMBEDDING_SIZE = embedding_size
        MAX_SEQ_LENGTH = max_seq_length
        embedding_weights = embedding_weights
        NUM_CLASSES = num_classes

        lstm_model = Sequential()
        lstm_model.add(
            Embedding(
                input_dim=VOCABULARY_SIZE,
                output_dim=EMBEDDING_SIZE,
                input_length=MAX_SEQ_LENGTH,
                weights=[embedding_weights],
                trainable=True
            )
        )
        lstm_model.add(LSTM(64, return_sequences=True))
        lstm_model.add(TimeDistributed(Dense(NUM_CLASSES, activation='softmax')))
        lstm_model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
        lstm_model.summary()
        return lstm_model

### Peso dos Embeddings

O modelo precisa de um parametro com os pesos dos embeddings, por isso foi criada uma
função para dar pesos a eles.

In [49]:
def emb_weights(word2vec, word_tokenizer, vocabulary_size, embedding_size):
    VOCABULARY_SIZE = vocabulary_size
    EMBEDDING_SIZE = embedding_size
    word2id = word_tokenizer.word_index
    embedding_weights = np.zeros((VOCABULARY_SIZE, EMBEDDING_SIZE))
    for word, index in word2id.items():
        try:
            embedding_weights[index, :] = word2vec[word]
        except KeyError:
            pass
    return embedding_weights


### Gerando o modelo

In [32]:
lstm = LSTMModel(
    vocabulary_size=vocabulary_size,
    embedding_size=embedding_size,
    max_seq_length=max_length,
    embedding_weights=emb_weights(
        word2vec=word2vec,
        word_tokenizer=data_extractor.word_tokenizer,
        vocabulary_size=vocabulary_size,
        embedding_size=embedding_size
    ),
    num_classes=data_extractor.Y_one_hot.shape[2]
)


<ipython-input-29-b23a19a0da47>:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  embedding_weights[index, :] = word2vec[word]


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 248, 128)          7668608   
_________________________________________________________________
lstm (LSTM)                  (None, 248, 64)           49408     
_________________________________________________________________
time_distributed (TimeDistri (None, 248, 24)           1560      
Total params: 7,719,576
Trainable params: 7,719,576
Non-trainable params: 0
_________________________________________________________________


### Treinando o modelo

In [50]:
epochs = 40
lstm_file = f'lstm_models/lstm_v3_embbedings_{epochs}e_non_prop'

In [ ]:
lstm.fit(
        train_test_and_validation.X_train,
        train_test_and_validation.Y_train,
        batch_size=128,
        epochs=epochs,
        validation_data=(train_test_and_validation.X_validation, train_test_and_validation.Y_validation)
)
lstm.save(lstm_file)

Epoch 1/40
257/257 [==============================] - 128s 483ms/step - loss: 0.0035 - accuracy: 0.9990 - val_loss: 0.0436 - val_accuracy: 0.9914

### Averiguando a efetividade do modelo

In [51]:
from keras.models import load_model
lstm = load_model(lstm_file)
lstm.evaluate(
    train_test_and_validation.X_test,
    train_test_and_validation.Y_test,
    verbose=1
)

322/322 [==============================] - 8s 23ms/step - loss: 0.0438 - accuracy: 0.9913

[0.04378342255949974, 0.991278886795044]

### Funções para imprimir tabelas

In [36]:
def overall_accuracy(cm):
    hits = np.diag(cm[1:, 1:]).sum()
    all_data = cm[1:, 1:].sum() + cm[0, 1:].sum()
    print("Hits: {}".format(hits))
    print("Total Data: {}".format(all_data))
    print("Percentage Accuracy: {:.2f}%".format((hits / all_data) * 100))
    print("\n")


def overall_accuracy_by_class(cm, tags):
    from texttable import Texttable

    diag_values = np.diag(cm / cm.astype(np.float).sum(axis=1))
    data_list = [['Tag', 'Value (%)']]
    for index, result in enumerate(diag_values):
        data_list.append([tags[index], "{:.2f}".format(result * 100)])
    t = Texttable(1000)
    t.add_rows(data_list)
    t.set_cols_align(["l", "r"])
    print(t.draw())


def print_confusion_matrix(cm, tags):
    from texttable import Texttable

    len_cm = len(cm)
    cm_aux = cm / cm.astype(np.float).sum(axis=1) * 100
    tags_names = ['Tags'] + [tags[index] for index in range(len_cm)]
    t = Texttable(1000)
    data_list = [tags_names]
    for index, result in enumerate(cm_aux):
        data_list.append([tags[index]] + list(cm_aux[index]))
    t.add_rows(data_list)
    print(t.draw())

### Função para transformar o OneHot em um array

In [37]:
def pred2array(sequences):
    token_sequences = []
    for categorical_sequence in sequences:
        for categorical in categorical_sequence:
            max_value_index = np.argmax(categorical)
            token_sequences.append(max_value_index)
    return np.array(token_sequences)

pred = lstm.predict(
        train_test_and_validation.X_test
    )
y_test_array = pred2array(train_test_and_validation.Y_test)
pred_array = pred2array(pred)

cm = confusion_matrix(
        y_true=y_test_array,
        y_pred=pred_array
)

### Acurácia Geral (ignorando os PAD)

In [38]:
overall_accuracy(cm)

Hits: 211314
Total Data: 233279
Percentage Accuracy: 90.58%




### Acurácia por classe

In [39]:
overall_accuracy_by_class(cm, data_extractor.tag_tokenizer.index_word)

+------------+-----------+
|    Tag     | Value (%) |
+============+===========+
| pad        |       100 |
+------------+-----------+
| n          |    92.690 |
+------------+-----------+
| prep       |    95.230 |
+------------+-----------+
| art        |    95.090 |
+------------+-----------+
| pu         |    99.980 |
+------------+-----------+
| nprop      |    81.160 |
+------------+-----------+
| v          |    90.420 |
+------------+-----------+
| adj        |    86.250 |
+------------+-----------+
| adv        |    83.020 |
+------------+-----------+
| kc         |    95.830 |
+------------+-----------+
| pcp        |    85.960 |
+------------+-----------+
| proadj     |    90.120 |
+------------+-----------+
| num        |    78.370 |
+------------+-----------+
| vaux       |    56.590 |
+------------+-----------+
| propess    |    91.430 |
+------------+-----------+
| ks         |    76.130 |
+------------+-----------+
| pro-ks-rel |    80.480 |
+------------+-----------+
|

### Matriz de Confusão

In [40]:
print_confusion_matrix(cm, data_extractor.tag_tokenizer.index_word)

+------------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+--------+------------+--------+--------+--------+--------+------------+--------+--------+
|    Tags    |  pad  |   n    |  prep  |  art   |   pu   | nprop  |   v    |  adj   |  adv   |   kc   |  pcp   | proadj |  num   |  vaux  | propess |   ks   | pro-ks-rel | prosub |  pden  |  cur   | pro-ks | adv-ks-rel |   in   | adv-ks |
+============+=======+========+========+========+========+========+========+========+========+========+========+========+========+========+=========+========+============+========+========+========+========+============+========+========+
| pad        | 100   | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0       | 0      | 0          | 0      | 0      | 0      | 0      | 0          | 0      | 0      |
+------------+-------+--------+--------+----

# [Parte 5] Resultados e Conclusões

Como podemos notar, o nosso modelo teve uma precisão de 90,58%, acima do foi encontrado
usando as bibliotecas do nltk. Esse valor é desconsiderando os PADs que foram adicionados
para padronizar. Esse valor foi acima do modelo de Trigram do NLTK, com uma validação em média
de 85%.

Olhando as classes individualmente, notamos que a precisão varia bastante entre classes

Nomes e Verbos tiveram uma alta precisão (93% e 90%, respectivamente), enquanto que nomes
próprios e verbos auxiliares mais baixos (81% e 57%, respectivamente). Nota-se que houve uma
tendência em trocar bastante esses valores na hora de classificar (verbo com verbos auxiliares e vice-versa),
provavelmente melhorar as camadas do modelo aqui desenvolvido.

As classes piores classificadas são classes que normalmente pessoas tem dificuldade em
diferenciar naturalmente, porque são palavras mais genéricas em suas funções (como a palavra que),
dependendo bastante do contexto e provavelmente o embeddings não foi capaz de determinar esse contexto
tão bem com o Skip-gram.

Ademais, o trabalho foi bastante interessante e desafiador, pois não é uma tarefa fácil
classificar palavras em classes gramaticais e desenvolver o modelo com acurácia
relativamente rendeu bastantes testes.